# F1-score as IAA measure

The average F1-score among pairs of annotators is numerically identical to the average positive specific agreement among annotators (kappa approaches these measures as the number of negative cases grow large).<sup>1</sup>

<sup>1</sup>Hripcsak, G., & Rothschild, A. S. (2005). Agreement, the F-measure, and reliability in information retrieval. *Journal of the American Medical Informatics Association : JAMIA, 12*(3), 296–298. https://doi.org/10.1197/jamia.M1733

In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from itertools import permutations, combinations

import sys
sys.path.insert(0, '..')
from utils.config import PATHS
from utils.data_process import drop_disregard, fix_week_14, pad_sen_id

In [2]:
pd.set_option('max_columns', None)

## Load and pre-process data

In [3]:
# load annotations

path = PATHS.getpath('data') / 'iaa'

df = pd.concat([pd.read_pickle(fp) for fp in path.glob('*_parsed.pkl')], ignore_index=True)

In [4]:
# load batch info

path = PATHS.getpath('data_to_inception_conll')

info = pd.DataFrame()

for batch in df.batch.unique():
    batch_info = pd.read_pickle(path / f'{batch}.pkl').assign(
        batch = batch,
    )
    info = info.append(batch_info, ignore_index=True)

In [5]:
# select IAA files

iaa = info.query("samp_meth == 'kwd_iaa'").NotitieID.unique()
df = df.loc[df.NotitieID.isin(iaa)]

In [6]:
# process annotations

domains = ['ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM']
other = ['target', 'background', 'plus']

df = df.assign(
    background_sent = lambda df: df.groupby('sen_id').background.transform('any'),
    target_sent = lambda df: df.groupby('sen_id').target.transform('any'),
    disregard_note = lambda df: df.groupby('NotitieID').disregard.transform('any'),
    pad_sen_id = df.sen_id.apply(pad_sen_id),
).pipe(fix_week_14)

df[domains + other] = df[domains + other].fillna(False)

In [7]:
# replace annotaotr names with alias (first letter)

df.annotator = df.annotator.str[0]

## Create sentence-level multi-label of all domains

In [8]:
# select notes that were annotated by all 6 annotators
# (needed since some annotators skipped some of the batches);
# create a sentence-level multi-label of all the domains

sent_labels = df.groupby(['annotator', 'pad_sen_id'])[domains].any().astype(int).apply(lambda s: s.to_list(), axis=1).rename('domain_labels').unstack(0).dropna()

## Pairwise metrics

In [9]:
pairs = list(permutations(sent_labels.columns, r=2))
combis = list(combinations(sent_labels.columns, r=2))
len(pairs)

30

In [10]:
def make_pairwise_classreport(y1, y2):
    return [
        precision_score(y1, y2),
        recall_score(y1, y2),
        f1_score(y1, y2)
    ]

In [11]:
cols = [
    'annotator1',
    'annotator2',
    'domain',
    'support',
    'precision',
    'recall',
    'f1_score',
]
classreport = pd.DataFrame(columns=cols)
for annotator1, annotator2 in pairs:
    for i, domain in enumerate(domains):
        ys = sent_labels[[annotator1, annotator2]].applymap(lambda x: x[i])
        data=[
            annotator1,
            annotator2,
            domain,
            ys[annotator1].sum(),
            *make_pairwise_classreport(ys[annotator1], ys[annotator2]),
        ]
        s = pd.Series(data, index=cols)
        classreport = classreport.append(s, ignore_index=True)

## Average F1-score per domain

In [12]:
# calculate average F1-score per domain
# and the min and max support

classreport.groupby('domain').agg({
    'f1_score': lambda s: round(s.mean(), 2),
    'support': lambda s: [s.min(), s.max()],
}).T

domain,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM
f1_score,0.64,0.58,0.42,0.66,0.45,0.78,0.34,0.62,0.57
support,"[21, 36]","[1, 4]","[1, 5]","[9, 18]","[7, 25]","[12, 17]","[2, 21]","[3, 6]","[10, 18]"


## F1-score per domain per pair

In [13]:
table = classreport.set_index(['annotator1', 'annotator2']).loc[combis, ['domain', 'f1_score']].set_index('domain', append=True).unstack(-1).droplevel(0, axis=1).reset_index().rename_axis('', axis=1)
table.loc[15,'ADM':] = table.loc[:,'ADM':].mean()
table.loc[15,'annotator1'] = 'mean'
table.loc[15,'annotator2'] = ''

table.style.background_gradient(cmap='Greens', axis=None).format({domain:"{:.2}" for domain in domains}).apply(lambda s: len(s) * ["background: black; color: white"],subset=15)

,annotator1,annotator2,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM
0,a,k,0.65,0.0,0.33,0.8,0.51,0.69,0.3,0.91,0.5
1,a,m,0.75,0.57,0.5,0.62,0.53,0.87,0.43,0.67,0.53
2,a,o,0.63,0.33,0.25,0.64,0.39,0.85,0.087,0.6,0.72
3,a,s,0.62,0.75,0.57,0.57,0.53,0.96,0.48,0.67,0.45
4,a,v,0.61,0.57,0.29,0.7,0.54,0.87,0.61,0.73,0.55
5,k,m,0.65,0.5,0.5,0.59,0.45,0.6,0.36,0.73,0.57
6,k,o,0.56,0.67,0.5,0.7,0.36,0.62,0.25,0.44,0.61
7,k,s,0.63,0.4,0.67,0.63,0.41,0.72,0.44,0.75,0.55
8,k,v,0.62,0.5,0.0,0.76,0.31,0.6,0.44,0.8,0.65
9,m,o,0.67,0.8,0.67,0.62,0.38,0.8,0.11,0.6,0.64


## Confusion between labels

In [14]:
def assign_result(df, dom, ann1, ann2):
    """
    Assign result to each row of `df` for domain in `dom` and annotators `ann1` and `ann2`. Labels of `ann1` are taken as gold and compared to `ann2`. If both are equal then it's a true or false positive. If `ann1` == True and `ann2` == False then it's a false negative if and only if `ann2` did not label anything else incorrectly. Else it's a confusion.

    parameters
    ----------
    df : pd.DataFrame
    dom : int
        index of the column to compare
    ann1 : str
        annotator 1 (used as gold)
    ann2 : str
        annotator 2

    returns
    -------
    pd.Series
        Series of len(df)
    """
    check_if_items_are_equal = lambda s: [i==j for i,j in zip(s[ann1], s[ann2])]
    df = df[[ann1, ann2]].assign(
        comp_all = lambda df: df.apply(check_if_items_are_equal, axis=1),
        comp_dom = lambda df: df.comp_all.str[dom],
        ann1_dom = lambda df: df[ann1].str[dom].astype(bool),
        ann2_sum = lambda df: df[ann2].apply(sum),
        n_diff = lambda df: df.comp_all.apply(lambda l: sum(not i for i in l)),
        result = pd.NA,
    )
    s = df.result.mask(df.comp_dom & df.ann1_dom, 'true positive')
    s = s.mask(s.isna() & df.comp_dom & ~df.ann1_dom, 'true negative')
    s = s.mask(s.isna() & ~df.comp_dom & ~df.ann1_dom, 'false positive')
    s = s.mask(s.isna() & ~df.ann2_sum, 'false negative')
    s = s.mask(s.isna() & (df.n_diff == 1), 'false negative')
    s = s.mask(s.isna(), 'confusion')
    return s

In [15]:
conf = sent_labels.copy()
for ann1, ann2 in combis:
    for i, domain in enumerate(domains):
        colname = f"{ann1}{ann2}_{domain}"
        conf = conf.assign(**{colname: assign_result(conf, i, ann1, ann2)})

In [16]:
conf.iloc[:,6:].loc[conf.applymap(lambda i: i=='confusion').any(axis=1)]

annotator,ak_ADM,ak_ATT,ak_BER,ak_ENR,ak_ETN,ak_FAC,ak_INS,ak_MBW,ak_STM,am_ADM,am_ATT,am_BER,am_ENR,am_ETN,am_FAC,am_INS,am_MBW,am_STM,ao_ADM,ao_ATT,ao_BER,ao_ENR,ao_ETN,ao_FAC,ao_INS,ao_MBW,ao_STM,as_ADM,as_ATT,as_BER,as_ENR,as_ETN,as_FAC,as_INS,as_MBW,as_STM,av_ADM,av_ATT,av_BER,av_ENR,av_ETN,av_FAC,av_INS,av_MBW,av_STM,km_ADM,km_ATT,km_BER,km_ENR,km_ETN,km_FAC,km_INS,km_MBW,km_STM,ko_ADM,ko_ATT,ko_BER,ko_ENR,ko_ETN,ko_FAC,ko_INS,ko_MBW,ko_STM,ks_ADM,ks_ATT,ks_BER,ks_ENR,ks_ETN,ks_FAC,ks_INS,ks_MBW,ks_STM,kv_ADM,kv_ATT,kv_BER,kv_ENR,kv_ETN,kv_FAC,kv_INS,kv_MBW,kv_STM,mo_ADM,mo_ATT,mo_BER,mo_ENR,mo_ETN,mo_FAC,mo_INS,mo_MBW,mo_STM,ms_ADM,ms_ATT,ms_BER,ms_ENR,ms_ETN,ms_FAC,ms_INS,ms_MBW,ms_STM,mv_ADM,mv_ATT,mv_BER,mv_ENR,mv_ETN,mv_FAC,mv_INS,mv_MBW,mv_STM,os_ADM,os_ATT,os_BER,os_ENR,os_ETN,os_FAC,os_INS,os_MBW,os_STM,ov_ADM,ov_ATT,ov_BER,ov_ENR,ov_ETN,ov_FAC,ov_INS,ov_MBW,ov_STM,sv_ADM,sv_ATT,sv_BER,sv_ENR,sv_ETN,sv_FAC,sv_INS,sv_MBW,sv_STM
pad_sen_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
160965810_0026,true negative,true negative,true negative,true negative,true negative,true negative,true negative,false negative,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,confusion,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true positive,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,confusion,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,confusion,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,confusion,true negative,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,confusion,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,confusion,true negative,true negative,true negative,true negative,true negative,true positive,true negative,true negative,true negative,true negative
269679147_0024,true negative,false negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,confusion,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true negative,confusion,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true negative,false negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,false positive,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,true negative,

In [17]:
pad_sen_id = '443067004_0046'

In [18]:
sent_labels.loc[pad_sen_id]

annotator
a    [0, 0, 0, 0, 0, 1, 1, 0, 0]
k    [0, 0, 0, 0, 1, 0, 1, 0, 0]
m    [0, 0, 0, 0, 0, 1, 1, 0, 0]
o    [0, 0, 0, 0, 0, 1, 0, 0, 0]
s    [0, 0, 0, 0, 0, 1, 1, 0, 0]
v    [0, 0, 0, 0, 0, 1, 1, 0, 0]
Name: 443067004_0046, dtype: object

In [19]:
query = "(pad_sen_id == @pad_sen_id) & (annotator == 'a')"
print(' '.join(df.query(query).token.to_list()))
print()

Dhr kan kwartiertje lopen .

